In [1]:
"""
Protein-Ligand Binding Affinity Analysis Script (Time-stamped)
------------------------------------------------
Description:
    This script automates the analysis of Microscale Thermophoresis (MST) 
    or similar binding affinity data. It processes CSV files in the current 
    directory, calculates binding kinetics using a one-site binding model, 
    and generates fitted plots.

    * Feature: Creates a unique output folder based on current time (HH:MM:SS)
      to prevent overwriting previous results.

    If outliers are marked in the raw data (column 'Is Outlier'), 
    the script performs analysis twice:
    1. Clean Data (Outliers excluded) - Primary Analysis
    2. All Data (Outliers included) - For validation/transparency

Author: [hanbyeonggu@gmail.com/BCK Lab]
Date: 2025-12-09
Dependencies: pandas, matplotlib, numpy, scipy
"""

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from scipy.optimize import curve_fit
import os
import glob
import datetime
import warnings

# Suppress runtime warnings (e.g., division by zero during optimization steps)
warnings.filterwarnings("ignore")

# ---------------------------------------------------------
# [Configuration] Parameters & Constants
# ---------------------------------------------------------
INPUT_FILE_EXT = "*.csv"
MAX_FILES_TO_PROCESS = 100
DELIMITER = ';'  # Adjust if your CSV uses commas (',')

# Column Mapping (Adjust these if raw data headers change)
COL_CONC = 'Ligand Concentration [M]'
COL_FNORM = 'Fnorm [‰]'
COL_OUTLIER = 'Is Outlier'

# ---------------------------------------------------------
# [Functions] Mathematical Models & Plotting
# ---------------------------------------------------------

def one_site_binding_model(x, kd, bottom, top):
    """
    Standard Langmuir isotherm equation for 1:1 binding.
    Formula: y = bottom + (top - bottom) * (x / (Kd + x))
    """
    return bottom + (top - bottom) * (x / (kd + x))

def perform_fitting_and_plotting(df_stats, save_path, label_suffix=""):
    """
    Performs non-linear least squares fitting and generates a publication-quality plot.
    """
    x_data = df_stats['Conc'].values
    y_data = df_stats['Mean'].values
    y_err = df_stats['Std'].values

    # 1. Parameter Estimation (Levenberg-Marquardt algorithm)
    try:
        # Dynamic initial guess for robustness:
        # Kd guess = median of concentrations
        # Bottom/Top guess = min/max of observed signals
        p0_guess = [np.median(x_data), np.min(y_data), np.max(y_data)]
        
        popt, pcov = curve_fit(one_site_binding_model, x_data, y_data, p0=p0_guess, maxfev=10000)
        kd_val, bottom_val, top_val = popt
        
        # Calculate R-squared (R2)
        residuals = y_data - one_site_binding_model(x_data, *popt)
        ss_res = np.sum(residuals**2)
        ss_tot = np.sum((y_data - np.mean(y_data))**2)
        r2 = 1 - (ss_res / ss_tot)
        fit_success = True
    except Exception as e:
        print(f"    [!] Fitting failed for {label_suffix}: {e}")
        kd_val, r2 = 0, 0
        fit_success = False

    # 2. Plotting
    plt.rcParams['font.family'] = 'sans-serif'
    plt.rcParams['font.sans-serif'] = ['Arial', 'Helvetica']
    plt.rcParams['font.size'] = 12
    
    fig, ax = plt.subplots(figsize=(6, 5))

    # Plot Raw Data points with Error bars
    ax.errorbar(x_data, y_data, yerr=y_err, fmt='o', color='black', ecolor='gray', 
                capsize=4, label='Experimental Data', zorder=3)

    # Plot Fit Curve
    if fit_success:
        x_fit = np.logspace(np.log10(min(x_data)/2), np.log10(max(x_data)*2), 1000)
        y_fit = one_site_binding_model(x_fit, *popt)
        ax.plot(x_fit, y_fit, color='#D32F2F', linewidth=2.5, label='Langmuir Fit', zorder=2)
        
        # Annotation Box
        txt = f'$K_d$ = {kd_val*1e9:.1f} nM\n$R^2$ = {r2:.4f}'
        ax.text(0.95, 0.95, txt, transform=ax.transAxes, verticalalignment='top', 
                horizontalalignment='right', 
                bbox=dict(boxstyle='round,pad=0.4', facecolor='white', alpha=0.9, edgecolor='lightgray'))

    ax.set_xscale('log')
    ax.set_xlabel('Ligand Concentration [M]', fontweight='bold')
    ax.set_ylabel('Fnorm [‰]', fontweight='bold')
    
    plt.tight_layout()
    
    # Save Figures
    plt.savefig(f"{save_path}.png", dpi=300)
    plt.savefig(f"{save_path}.svg", format='svg')
    plt.close()
    print(f"    -> Graph Saved: {os.path.basename(save_path)}.png")

# ---------------------------------------------------------
# [Main] Batch Processing Logic
# ---------------------------------------------------------
def main():
    # 1. Setup Output Directory with Timestamp (Prevents Overwriting)
    # Format: Analysis_MMDD_HHMMSS (e.g., Analysis_1209_143005)
    timestamp_str = datetime.datetime.now().strftime("%m%d_%H%M%S")
    output_dir = os.path.join(os.getcwd(), f"Analysis_{timestamp_str}")
    
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
        print(f">>> Created Output Directory: {output_dir}")
    
    # 2. Find CSV Files
    file_list = glob.glob(INPUT_FILE_EXT)
    file_list = [f for f in file_list if "Processed" not in f] # Avoid re-processing output files
    
    if not file_list:
        print(">>> No CSV files found in the current directory.")
        return

    print(f">>> Found {len(file_list)} files. Processing up to {MAX_FILES_TO_PROCESS}...")

    # 3. Process Files
    for i, filepath in enumerate(file_list[:MAX_FILES_TO_PROCESS]):
        filename = os.path.basename(filepath)
        base_name = os.path.splitext(filename)[0]
        print(f"\n[{i+1}] Processing: {filename}")

        try:
            df = pd.read_csv(filepath, delimiter=DELIMITER)
        except Exception as e:
            print(f"    [Error] Could not read file: {e}")
            continue

        # Validation: Check Columns
        if COL_CONC not in df.columns or COL_FNORM not in df.columns:
            print(f"    [Skip] Missing essential columns.")
            continue

        # -----------------------------------------------------
        # Analysis Workflow
        # -----------------------------------------------------
        
        # Check if Outliers exist in the dataset
        has_outliers = False
        if COL_OUTLIER in df.columns:
            if df[COL_OUTLIER].eq(True).any() or df[COL_OUTLIER].eq(1).any():
                has_outliers = True

        datasets_to_process = []

        # A. Prepare Clean Data (Standard)
        if COL_OUTLIER in df.columns:
            df_clean = df[df[COL_OUTLIER] != True] # Exclude outliers
        else:
            df_clean = df # No outlier column, treat all as clean
        datasets_to_process.append(('Clean', df_clean))

        # B. Prepare All Data (Only if outliers detected)
        if has_outliers:
            datasets_to_process.append(('AllData_with_Outliers', df))
            print("    -> [Info] Outliers detected. Comparing 'Clean' vs 'All Data'.")

        # C. Iterate through datasets (Clean vs All), Save CSV, and Plot
        for label, data_subset in datasets_to_process:
            if data_subset.empty:
                continue

            # 1. Calculate Statistics (Mean & Std per concentration)
            stats = data_subset.groupby(COL_CONC)[COL_FNORM].agg(['mean', 'std']).reset_index()
            stats.columns = ['Conc', 'Mean', 'Std']
            
            # 2. Save Processed Data to CSV (For transparency/submission)
            save_name = f"{base_name}_{label}_Processed.csv"
            save_path_csv = os.path.join(output_dir, save_name)
            stats.to_csv(save_path_csv, index=False)
            
            # 3. Generate Graph
            save_path_plot = os.path.join(output_dir, f"{base_name}_{label}_Plot")
            perform_fitting_and_plotting(stats, save_path_plot, label_suffix=label)

    print(f"\n>>> Batch processing completed. Results saved in: {output_dir}")

if __name__ == "__main__":
    main()